# 최종 정리

## 요구사항

이 것을 왜 하냐? 계속 구글 스프레스 시트로 반복 작업을 하고 싶지 않았기 때문이다.

- Ajax 로 특정 시군구 긁어오기
- 정렬, 복사, 붙여넣기 등
- 스프레트 시트에 지역-탭 별로 나누기

이 정보의 최종 활용자는? 50대 컴퓨터에 익숙치 않은 어른. 그가 이 프로그램을 조작하진 않는다. 그리고 난 그에게 엑셀 파일만 주면 된다.

몇 개의 필드를 숨기거나 하는 작업을 해도 된다. 그렇지만, 자료를 업데이트할 때마다 매번 자료의 형태를 가공해야하는 작업이 싫다.

즉, 최종적으로 나는

1. 내가 원하는 지역 경기도, 인천, 서울의 모든 아파트 정보를
    - 각 페이지에
    - 함수 한 번으로 가져오고
    - 저장한 다음에
    - 이것을 아래의 형태로 가공해서 각 부천,시흥시,광명시,구로구,양천구,부평구 페이지에 저장한다.
    - 양식을 붙여넣을 수 있기 때문에.

2. 최신 정보를 갱신할 때는.
    - 기존의 정보의 명칭과 새로 받아온 것의 명칭을 비교해서
    - 정보가 틀릴 때는 최신 것으로 업데이트하고
    - 정보가 없을 때는 새로 추가하며
    - 정보가 같을 때는 아무일도 하지 않는다.

그렇게 되면 나는 이미 가공된 자료의 필드 몇개를 숨기거나 양식을 붙여넣는 수준으로 작업을 편하게 할 수 있다.

연번|명칭|법정동|난방|관리사무실|팩스|준공일|방식|연차|동수|방광|요일|시간|금액|선입|문어|매수|1차|2차|비고
--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--
ㅇ|

## 피드백

비판과 재비판을 함 해보려고 한다.

**원하는 지역의 아파트 코드만 가져온다**

1. 필요한 건 오직 특정 시군구 정보 뿐이다.
 - 아니다. 특정 시군구 법정코드와 아파트 코드가 필요하다.
2. 특정 시군구 아파트 코드만 필요하다
 - 가능하려면 2.x 메가짜리 파일에서 특정 동을 검색해서 그부분만 잘라서 xml 로 만든 뒤에 매번 csv 임포트 해줘야 한다.

즉, sql 에 현재 가진 모든 법정코드와 주소를 테이블로 넣어두고 아파트 정보가 디테일하게 담긴 테이블에 정보를 넣어두는 것이 좋다.

**아파트 코드만 얻어오면 아파트 정보를 얻을 수 있다.**

- 그러나 읽어온 아파트 정보는 처리가 필요하다.
- 게다가 영문으로 된 정보가 한글로 무엇인지 번역될 필요가 있다.

```
http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255
```

In [2]:
  import ( "io/ioutil";"net/http";"strings";"fmt";"bytes" )

    res, err := http.Get("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255")
    defer res.Body.Close()
    body, err := ioutil.ReadAll(res.Body); buf := bytes.NewBuffer(body); json := buf.String(); fmt.Println(json[:300])
  	

{"resultMap_match":{"KAPT_CODE":"A10027255","TOWN_CODE":20306348},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"-","SUBWAY_LINE":"7호선, 인천선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"분무식","KAPT_PE5AREA":0,"KAP
301
<nil>


# 구현 

## 모든 법정동 코드 정보를 테이블에 넣기

### 파일을 읽어서 존재하는 정보만 배열로 준비하기

1. 존재하는 법정동 정보 중
2. 코드 뒤 두자리가 00이면서
3. 이름이 00동인 것만 추려서
4. 코드 앞 8자리와 법정동명을 추출한다.

In [1]:
import ("encoding/csv"; "fmt"; "os"; "github.com/kniren/gota/dataframe";
       "bytes";"regexp";"strings")

// 접속 생성 및  닫기 지연 걸기
 csvFile, err := os.Open("./법정동코드_전체자료.txt")
 defer csvFile.Close()

// 일단 리더생성 후 쉼표 옵션 기호를 탭으로 설정
 reader := csv.NewReader(csvFile)
 reader.Comma = '\t' // Use tab-delimited instead of comma <---- here!
 reader.FieldsPerRecord = -1

// 모두 읽어제껴버림
 arrayData, err := reader.ReadAll()

// 헤더를 제외한 데이터만 배열에 담기
bjdArrData := [][]string{}
ind := 0

// 패턴 매칭 숫자 8개, 0 두개
r, _ := regexp.Compile("[0-9]{8}00")

for index, row := range arrayData{
    
    법정동코드 := row[0][:8]
    법정동명 := row[1]
    빈배열 := []string{법정동코드,법정동명 }
    존재여부 := row[2]
    숫자패턴 := r.MatchString(row[0])
    주소묶음 := strings.Fields(row[1])
    
                         
    if (index == 0){
        // bjdArrData = append(filteredArrData, row[:2])
        // index가 0 이면 다시 for 룹 시작으로 가서 index 1 부터 처리
        // 즉 헤더 빼고 데이터처리만 하라는 말
        continue
    }else if( 존재여부 == "존재" && 숫자패턴 && len(주소묶음) == 3 ){
        bjdArrData = append(bjdArrData, 빈배열)
    }
}

fmt.Println(bjdArrData[:50])

[[11110101 서울특별시 종로구 청운동] [11110102 서울특별시 종로구 신교동] [11110103 서울특별시 종로구 궁정동] [11110104 서울특별시 종로구 효자동] [11110105 서울특별시 종로구 창성동] [11110106 서울특별시 종로구 통의동] [11110107 서울특별시 종로구 적선동] [11110108 서울특별시 종로구 통인동] [11110109 서울특별시 종로구 누상동] [11110110 서울특별시 종로구 누하동] [11110111 서울특별시 종로구 옥인동] [11110112 서울특별시 종로구 체부동] [11110113 서울특별시 종로구 필운동] [11110114 서울특별시 종로구 내자동] [11110115 서울특별시 종로구 사직동] [11110116 서울특별시 종로구 도렴동] [11110117 서울특별시 종로구 당주동] [11110118 서울특별시 종로구 내수동] [11110119 서울특별시 종로구 세종로] [11110120 서울특별시 종로구 신문로1가] [11110121 서울특별시 종로구 신문로2가] [11110122 서울특별시 종로구 청진동] [11110123 서울특별시 종로구 서린동] [11110124 서울특별시 종로구 수송동] [11110125 서울특별시 종로구 중학동] [11110126 서울특별시 종로구 종로1가] [11110127 서울특별시 종로구 공평동] [11110128 서울특별시 종로구 관훈동] [11110129 서울특별시 종로구 견지동] [11110130 서울특별시 종로구 와룡동] [11110131 서울특별시 종로구 권농동] [11110132 서울특별시 종로구 운니동] [11110133 서울특별시 종로구 익선동] [11110134 서울특별시 종로구 경운동] [11110135 서울특별시 종로구 관철동] [11110136 서울특별시 종로구 인사동] [11110137 서울특별시 종로구 낙원동] [11110138 서울특별시 종로구 종로2가] [11110139 서울특별시 종로구 팔판동] [11110140 서울특별시 종로

### 테이블 생성 후 배열 자료를 추가하기 

생성

In [2]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Println(err)
}
defer db.Close()

droptable := `DROP TABLE IF EXISTS 법정동, 아파트정보;`

createtable:= `CREATE TABLE 법정동 (
                법정동코드 bigint NOT NULL DEFAULT 0,
                법정동명 varchar(30)  NOT NULL DEFAULT ''
              )`

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err1 := db.Prepare(createtable)
// 쿼리 날릴 준비를 하고 실행한다.
stmt2, err1 := db.Prepare(droptable)



_, err = stmt2.Exec()
if err != nil {
    fmt.Println(err.Error())
}
defer stmt2.Close()

_, err = stmt.Exec()
if err != nil {
    fmt.Println(err.Error())
}
defer stmt.Close()

벌크 임포트(대량 가져오기) https://godoc.org/github.com/lib/pq#hdr-Bulk_imports

prepared statement http://go-database-sql.org/prepared.html


In [3]:
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("법정동", "법정동코드", "법정동명"))
if err != nil {
	fmt.Println(err)
}

for _, bjd := range bjdArrData {
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
        _, err = stmt.Exec(bjd[0], bjd[1])
	if err != nil {
		fmt.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}

## 특정 키워드를 입력하면 해당하는 모든 법정동 코드 가져오는 함수 만들기

### 함수 설계 (mock)

In [33]:
func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    
    법정동코드결과 = append(법정동코드결과, 1001, 1002, 1003)
    
    return 법정동코드결과
}

fmt.Println(getBjd("지역키워드"))

[1001 1002 1003]
17
<nil>


### 쿼리 설계

In [36]:
// db 접속 포인터를 얻고
db, err := sql.Open("postgres", dbinfo)

if err != nil { log.Println(err)}
defer db.Close()

// Query the database.
rows, err := db.Query(`
    SELECT 법정동코드
    FROM 법정동
    WHERE 법정동명 LIKE $1
    `, "%서울%")

if err != nil { log.Println(err)}
defer rows.Close()
    
fmt.Println("질의 날리기")

법정동코드결과 := []string{}

for rows.Next() {
    
    코드하나 := ""

if err := rows.Scan(&코드하나); err != nil {
log.Println(err)}
 
   법정동코드결과 =  append(법정동코드결과,코드하나)
// fmt.Printf("%.2d, \n", 법정동코드)
}
    
fmt.Println("출력하기", 법정동코드결과[:4])
    

질의 날리기
출력하기 [1100000000 1111000000 1111010100 1111010200]
59
<nil>


### 지역 키워드에 따라 법정동 코드를 가져오는 함수 완성

In [4]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

func getBjd(areaName string) [][]string {
    
    법정동코드결과 := [][]string{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    db, err := sql.Open("postgres", dbinfo)
    if err != nil { log.Println(err)}
    defer db.Close()
    
    // Query the database.
    rows, err := db.Query(`SELECT 법정동코드,법정동명 FROM 법정동 WHERE 법정동명 LIKE $1`,
                      "%"+ areaName+"%")
    index := 0
    for rows.Next() {    
        코드하나 := ""
        이름하나 := ""
        if err := rows.Scan(&코드하나,&이름하나); err != nil {log.Println(err)}
        한줄 := []string{코드하나,이름하나}
        법정동코드결과 =  append(법정동코드결과, 한줄)
        index += 1
    }
    
    return 법정동코드결과
}

## 법정동을 주면 아파트 코드를 가져오는 함수 작성

### 함수 설계 - Mock

In [5]:
func getAptCode(bjdCode int) []string {
    
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    아파트코드결과 = append(아파트코드결과, "A1001", "A1002", "A1003")
    
    return 아파트코드결과
}

fmt.Println(getAptCode(263802006002))

[A1001 A1002 A1003]
20
<nil>


### get 요청과 처리 설계

계속 timeout 오류가 나서 내가 뭘 잘못했나. Timeout 시간까지 늘렸는데... 집 네트워크가 이상해서 그랬다. 현재 lte 네트워크로 작업하니 슉슉 잘 됨.

#### 요청 주소 만들고 GET 호출하기

직접 Get 을 썼다가. 아래처럼 request 요청으로 바꿈.
http://golang.site/go/article/102-HTTP-GET-호출

고 기본 get 은 실패시 재시도 기능이 없다.
https://github.com/sethgrid/pester

In [5]:
import (
    "fmt"
    "log"
    "net/http"
    "os"
 "net"
  "net/url"
    "strconv"
    "time"
    "io/ioutil"
    "github.com/sethgrid/pester"
)

func getAptlistRsp(bjdCode string) []uint8{
    
    aptListurl := "http://www.k-apt.go.kr/kaptinfo/getKaptList.do"
    req, err := http.NewRequest("GET", aptListurl, nil)
    if err != nil {
        log.Print(err)
    }

    q := req.URL.Query()   
    q.Add("bjd_code",bjdCode) 
    req.URL.RawQuery = q.Encode()
    
    // url := req.URL.String()
    // fmt.Println(req.URL.String())
    
    
    // Client객체에서 Request 실행
//     client := &http.Client{Timeout: 40 * time.Second,}
//     resp, err := client.Do(req)
    
    // pester 사용
    resp, err := pester.Do(req)
    if err != nil {
        log.Println("error GETing example.com", err)
    }
    defer resp.Body.Close()
    log.Printf("example.com %s", resp.Status)
 
    // 결과 출력
    bytes, _ := ioutil.ReadAll(resp.Body)
    // str := string(bytes) //바이트를 문자열로
     // fmt.Println(str[:100])
    
    
    return  bytes
}

#### 주소 요청 결과로 구조체 만들기

구조는 다음과 같다.

In [ ]:
{
  "resultList": [
    {
      "KAPT_USEDATE": "199701",
      "BJD_CODE": "1135010500",
      "KAPT_NAME": "노원현대아파트",
      "OPEN_TERM": null, 
      "KAPT_CODE": "A13981604",
      "OCCU_FIRST_DATE": "199702",
      "X": 205515.315,
      "Y": 461795.5,
      "ENERGY_B_COUNT": 0,
      "BJD_NAME": "서울특별시 노원구 상계동"
    },
      ...
}

구조체는 다음과 같음..  

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/

네스티드로 구조체 짤 수도 있다.
https://stackoverflow.com/questions/32125816/parsing-xml-in-golang-unmarshaling

In [6]:
import(
    "encoding/json"
    "bytes"
"reflect")


type AptBasicResultList struct {
     결과목록 []AptBasicResult  `json:"resultList"`
}

type AptBasicResult struct{
    아파트사용일 string `json:"KAPT_USEDATE"`
    법정동코드 string  `json:"BJD_CODE"`
    아파트이름 string `json:"KAPT_NAME"`
    OPEN_TERM string `json:"OPEN_TERM"` // null 값을 타입으로 받을 수 있나?
    아파트코드 string `json:"KAPT_CODE"`
    완공첫날 int `json:"OCCU_FIRST_DATE"`
    x좌표 float32 `json:"X"`
    y좌표 float32 `json:"Y"`
    에너지_B_횟수 int `json:"ENERGY_B_COUNT"`
    법정동명 string  `json:"BJD_NAME"`
}


func unMarshalAptJson(responseBody []byte) AptBasicResultList {
    
    var data AptBasicResultList
    json.Unmarshal(responseBody, &data)

    fmt.Println(data.결과목록)
     jsonData, _ := json.Marshal(data)
     fmt.Println(string(jsonData))

 return data
}


#### XML 로부터 아파트 코드 추출하기

In [7]:
import(
    "encoding/json"
    "encoding/xml"
    "bytes"
"reflect"
"fmt")


func getAptCode(aptXmlData AptBasicResultList) []string {
        
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    items := aptXmlData.결과목록
    
    for _,v := range items{
        // fmt.Println(k,v)
        아파트코드결과 = append(아파트코드결과, v.아파트코드 )
    }
    // fmt.Println(아파트코드결과[:10])
    
    return 아파트코드결과
}


## 아파트 기본 정보 담을 테이블 생성하기

### 아파트 기본 정보 xml 정보 확인


response body 만 xml 로 바꿔서 xml 파싱한다.
https://tutorialedge.net/golang/parsing-xml-with-golang/

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/


받게되는 정보에 관한 설명
https://www.data.go.kr/subMain.jsp#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTIkQF5wdWJsaWNEYXRhRGV0YWlsUGs9dWRkaTphMTJjNmU3Ni04OTgyLTRlM2UtYWZlMi05NDQxMmM3ZDBhNzkkQF5tYWluRmxhZz10cnVl

In [8]:
import(
    "encoding/json"
    "encoding/xml"
    "io/ioutil"
    "bytes"
    "net/http"
    "reflect"
    "log"
    "fmt"
    "github.com/sethgrid/pester"
)

func getAptDetailtRsp(kaptCode string) []byte{
    
    aptinfourl := "http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do"
    req, err := http.NewRequest("GET", aptinfourl, nil)
    if err != nil {
        log.Print(err)
        //os.Exit(1)
    }

    q := req.URL.Query()  
    q.Add("kapt_code",  kaptCode)
    req.URL.RawQuery = q.Encode()
    // url := req.URL.String()
//     fmt.Println(req.URL.String())
//     return  url   
    
        // Client객체에서 Request 실행
//     client := &http.Client{Timeout: 40 * time.Second,}
//     resp, err := client.Do(req)
    
    // pester 사용
    resp, err := pester.Do(req)
    if err != nil {
        log.Println("error GETing example.com", err)
    }
    defer resp.Body.Close()
    log.Printf("example.com %s", resp.Status)
 
    // 결과 출력
    bytes, _ := ioutil.ReadAll(resp.Body)
    str := string(bytes) //바이트를 문자열로
    fmt.Println(str)
    
    
    return  bytes
    
}

In [9]:
type AptDetailResponse struct {
    매치정보 AptDetailItemMatch `json:"resultMap_match"`
    세부정보 AptDetailItem `json:"resultMap_kapt"`
    세대정보 []AptDetailSedae `json:"resultMap_kapt_areacnt"`
    주소정보 []AptDetailAddress`json:"resultMap_kapt_addrList"`
}

type AptDetailItemMatch struct {
    아파트코드 string `json:"KAPT_CODE"`
    마을코드 int `json:"TOWN_CODE"`
}

type AptDetailSedae struct {
    세대수 int `json:"KAPTDA_CNT"`
    전용면적구분 int `json:"AREA_GBN"`// 60㎡ 이하: 60㎡ ~ 85㎡ 이하: 85㎡ ~ 135㎡ 이하: 135㎡ 초과:
    아파트코드 string `json:"KAPT_CODE"`
}

type AptDetailAddress struct {
    법정동주소 string `json:"ADDR"`
    아파트코드 string `json:"KAPT_CODE"`
    주소구분 string `json:"ADDR_GBN"`
}

type AptDetailItem struct {
    아파트이름 	string `json:"KAPT_NAME"`
    분양형태	string `json:"CODE_SALE"`
    난방방식	string `json:"CODE_HEAT"`
    연면적	float64 `json:"KAPT_TAREA"`
    동수 string `json:"KAPT_DONG_CNT"`
    시공사 string `json:"KAPT_BCOMPANY"`
    시행사 string `json:"KAPT_ACOMPANY"`
    관리사무소연락처	string `json:"KAPT_TEL"`
    관리사무소팩스	string `json:"KAPT_FAX"`
    홈페이지주소	string `json:"KAPT_URL"`
    단지분류	string `json:"CODE_APT"`
    
    일반관리방식 string `json:"CODE_MGR"`
    일반관리인원 	string `json:"KAPT_MGR_CNT"`
    일반관리계약업체  	string `json:"KAPT_CCOMPANY"` 
    
    경비관리방식	string `json:"CODE_SEC"`
    경비관리인원	string `json:"KAPTD_SCNT"`
    경비관리계약업체 	string `json:"KAPTD_SEC_COM"`

    청소관리방식	string `json:"CODE_CLEAN"`
    청소관리인원	string `json:"KAPTD_CLCNT"`
    음식물처리방법 	string `json:"CODE_GARBAGE"`
    
    소독관리방식	string `json:"CODE_DISINF"`
    연간소독횟수 	string `json:"KAPTD_DCNT"`
    소독방법	string `json:"DISPOSAL_TYPE"`
    
    지상주차장대수 string `json:"KAPTD_PCNT"`
    지하주차장대수 string `json:"KAPTD_PCNTU"`
    
    cctv대수  string `json:"KAPTD_CCCNT"`
    부대복리시설  string `json:"WELFARE_FACILITY"`
    수전용량  string `json:"KAPTD_ECAPA"`
    전기안전관리자법정선임여부  	string `json:"CODE_EMGR"`
    급수방식  string `json:"CODE_WSUPPLY"`
    
        건물구조  string `json:"CODE_STR"`
    세대전기계약방식 string `json:"CODE_ECON"`
    화재수신반방식  string `json:"CODE_FALARM"`
    
    승강기관리형태  string `json:"CODE_ELEV"`
    // 승객용승강기  string `json:"KAPTD_ECNTP"`
    //화물용승강기  string `json:"KAPTD_PCNT"`
    //비상승강기 string `json:"KAPTD_DCNT"`

    주차관제홈네트워크  	string `json:"CODE_NET"`
    
    버스정류장 	string `json:"KAPTD_WTIMEBUS"`
    편의시설  	string `json:"CONVENIENT_FACILITY"`
    교육시설  	string `json:"EDUCATION_FACILITY"`
    
    복도유형	string `json:"CODE_HALL"`
    사용승인일	string `json:"KAPT_USEDATE"`
    주거전용면적 float64 `json:"KAPT_MAREA"`
    
    인근지하철역	string `json:"SUBWAY_STATION"`
    인근지하철호선	string `json:"SUBWAY_LINE"`
    지하철까지시간 	string `json:"KAPTD_WTIMESUB"`
    아파트코드	string `json:"KAPT_CODE"`
    
 }


func unMarshalAptDetail(responseBody []byte) AptDetailResponse {
    
    var data AptDetailResponse
    json.Unmarshal([]byte(responseBody), &data)
    
     // fmt.Println(data.세부정보.명칭)
    jsonData, _ := json.Marshal(data)
    fmt.Println(string(jsonData))

 return data
}

### 쿼리 작성 + 테이블 생성

In [10]:
createAptTableScript := 
`CREATE TABLE 아파트정보 (
              
    아파트코드 varchar(10)  NOT NULL DEFAULT '',
    아파트이름 	varchar(51)  NOT NULL DEFAULT '',
    마을코드 bigint NOT NULL DEFAULT 0,
    법정동주소 varchar(200) NOT NULL DEFAULT 0,
    도로명주소 varchar(200) NOT NULL DEFAULT 0,
    
    단지분류	varchar(51)  NOT NULL DEFAULT '',
    
    동수 varchar(101)  NOT NULL DEFAULT '',    
    세대수 bigint NOT NULL DEFAULT 0,
    
    전용면적60이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적60이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적60이상85이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적60이상85이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적85이상135이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적85이상135이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적135초과아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적135초과세대수  bigint NOT NULL DEFAULT 0,
    
    분양형태	varchar(52)  NOT NULL DEFAULT '',
    난방방식	varchar(52)  NOT NULL DEFAULT '',
    복도유형	varchar(52)  NOT NULL DEFAULT '',

    연면적	double precision  NOT NULL DEFAULT 0,

    시공사 varchar(102)  NOT NULL DEFAULT '',
    시행사 varchar(103)  NOT NULL DEFAULT '',
    
    관리사무소연락처	varchar(53)  NOT NULL DEFAULT '',
    관리사무소팩스	varchar(53)  NOT NULL DEFAULT '',
    홈페이지주소	varchar(104)  NOT NULL DEFAULT '',
    
    일반관리방식 varchar(54)  NOT NULL DEFAULT '',
    일반관리인원 varchar(52)  NOT NULL DEFAULT '',
    일반관리계약업체  	varchar(105)  NOT NULL DEFAULT '',
    
    경비관리방식	varchar(54)  NOT NULL DEFAULT '',
    경비관리인원	varchar(52)  NOT NULL DEFAULT '',
    경비관리계약업체 	varchar(106)  NOT NULL DEFAULT '',

    청소관리방식	varchar(54)  NOT NULL DEFAULT '',
    청소관리인원	varchar(52)  NOT NULL DEFAULT '',
    음식물처리방법 	varchar(107)  NOT NULL DEFAULT '',
    
    소독관리방식	varchar(54)  NOT NULL DEFAULT '',
    연간소독횟수  varchar(54)  NOT NULL DEFAULT '',
    소독방법	varchar(54)  NOT NULL DEFAULT '',
    
    승강기관리형태  	varchar(55)  NOT NULL DEFAULT '',
    승객용승강기 varchar(54)  NOT NULL DEFAULT '',
    화물용승강기 varchar(54)  NOT NULL DEFAULT '',
    비상승강기 varchar(54)  NOT NULL DEFAULT '',
    주차관제홈네트워크  	varchar(58)  NOT NULL DEFAULT '',

    건물구조  	varchar(108)  NOT NULL DEFAULT '',
    세대전기계약방식  	varchar(55)  NOT NULL DEFAULT '',
    화재수신반방식  	varchar(55)  NOT NULL DEFAULT '',

    
    지상주차장대수 varchar(57)  NOT NULL DEFAULT '',
    지하주차장대수 varchar(57)  NOT NULL DEFAULT '',
    
    cctv대수  	varchar(57)  NOT NULL DEFAULT '',
    부대복리시설  	varchar(70)  NOT NULL DEFAULT '',
    수전용량  	varchar(57)  NOT NULL DEFAULT '',
    전기안전관리자법정선임여부  	varchar(57)  NOT NULL DEFAULT '',
    급수방식  	varchar(57)  NOT NULL DEFAULT '',
    

    
    버스정류장 	varchar(51)  NOT NULL DEFAULT '',
    편의시설  	varchar(150)  NOT NULL DEFAULT '',
    교육시설  	varchar(150)  NOT NULL DEFAULT '',
    

    사용승인일	varchar(51)  NOT NULL DEFAULT '',
    주거전용면적 double precision  NOT NULL DEFAULT 0,
    
    인근지하철역	varchar(50)  NOT NULL DEFAULT '',
    인근지하철호선	varchar(50)  NOT NULL DEFAULT '',
    지하철까지시간 	varchar(50)  NOT NULL DEFAULT ''


)`

In [11]:

import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)



func createAptTable(script string){
    
_ = pg.Efatal
    
const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Print(err)
}
defer db.Close()

// droptable := `DROP TABLE IF EXISTS 법정동, 아파트정보;`

// stmt2, err1 := db.Prepare(droptable)
// _, err = stmt2.Exec()
// if err != nil {
//     fmt.Println(err.Error())
// }
// defer stmt2.Close()

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err := db.Prepare(script)

_, err = stmt.Exec()
if err != nil {
    fmt.Print(err.Error())
}
defer stmt.Close()
    }

## 특정 주소 키워드를 주면 지역 내 아파트정보를 반환하는 함수 만들기

In [12]:
import "strconv"

// 지역 키워드를 주면, 아파트 정보들을 반환한다.
func getAptBaseInfo(areaKeyword string) []AptDetailResponse {
    
    
  
    // bucket
    var aptInformations []AptDetailResponse
    
    
    fmt.Println("법정동 목록 시작","-----------")
    
    // 법정동 주소의 목록을 반환
    법정동목록 := getBjd(areaKeyword)
    
    fmt.Println(법정동목록)
    
    fmt.Println("법정동 목록 완료","-----------")
    
    // 법정동 하나씩 마다
    for _, bjdCode := range 법정동목록{

        bjdCode := bjdCode[0]

        fmt.Println(bjdCode,"아파트 목록 조회 시작","-----------")
        
        respListBody := getAptlistRsp(bjdCode)
        // _ = respBody
        xmlData := unMarshalAptJson(respListBody)
        // _ = xmlData
        
        fmt.Println("아파트 목록 조회 완료","-----------")
        
        fmt.Println("아파트 목록 마샬링 시작","-----------")
        
        // 여러 아파트 코드를 알아내서
        aptCodes := getAptCode(xmlData)

        fmt.Println("아파트 목록 마샬링 완료","-----------")
        
        // 아파트 코드 하나마다
        fmt.Println("아파트 정보 조회 시작","-----------")
        
        for index, kaptCode := range aptCodes{

            rspDetailBody := getAptDetailtRsp( kaptCode)
            xmlData := unMarshalAptDetail(rspDetailBody)
            
            // 정보를 알아낸 다음 차곡차곡 끼워 넣는다.
            aptInformations = append(aptInformations, xmlData)
            fmt.Println(  index+1,"/" ,len(aptCodes) ,xmlData.세부정보.아파트이름, "처리완료")
        }
        
        fmt.Println(len(aptInformations),"개 아파트 처리 완료")
    }

    return aptInformations
}


## 테이블에 자료를 넣고 원하는 것만 꺼내서 csv 파일로 출력하기

자료를 모두 담을 구조체 생성 // 이럴 바에야 json flatten 시키고 string 을 한 구조체에 담을 걸 ㅠㅠ

In [13]:
열이름 := []string{
"아파트코드",
"마을코드",
"법정동주소",
"도로명주소",
"세대수",
"전용면적60이하세대수",
"전용면적60이상85이하세대수",
"전용면적85이상135이하세대수",
"전용면적135초과세대수",
"전용면적60이하아파트코드",
"전용면적60이상85이하아파트코드",
"전용면적85이상135이하아파트코드",
"전용면적135초과아파트코드",
"아파트이름",
"분양형태",
"난방방식",
"연면적",
"동수",
"시공사",
"시행사",
"관리사무소연락처",
"관리사무소팩스",
"홈페이지주소",
"단지분류",
"일반관리방식",
"일반관리인원",
"일반관리계약업체",
"경비관리방식",
"경비관리인원",
"경비관리계약업체",
"청소관리방식",
"청소관리인원",
"음식물처리방법",
"소독관리방식",
"연간소독횟수",
"소독방법",
"건물구조",
"세대전기계약방식",
"화재수신반방식",
"승강기관리형태",
"지상주차장대수",
"지하주차장대수",
"cctv대수",
"부대복리시설",
"수전용량",
"전기안전관리자법정선임여부",
"급수방식",
"승객용승강기",
"화물용승강기",
"비상승강기",
"주차관제홈네트워크",
"버스정류장",
"편의시설",
"교육시설",
"복도유형",
"사용승인일",
"주거전용면적",
"인근지하철역",
"인근지하철호선",
"지하철까지시간",
}

```sql
매치 := apt.매치정보
세부 := apt.세부정보
세대1 := apt.세대정보[0]
세대2 := apt.세대정보[1]
세대3 := apt.세대정보[2]
세대4 := apt.세대정보[3]
주소 := apt.주소정보[0]

stmt.Exec(
    세부.아파트코드,
    매치.마을코드,
    주소.법정동주소,
    주소.주소구분,
    세대.세대수,
    세대1.세대수,
    세대2.세대수,
    세대3.세대수,
    세대4.세대수,
    세대1.아파트코드,
    세대2.아파트코드,
    세대3.아파트코드,
    세대4.아파트코드,
    세부.아파트이름,
    세부.분양형태,
    세부.난방방식,
    세부.연면적,
    세부.동수,
    세부.시공사,
    세부.시행사,
    세부.관리사무소연락처,
    세부.관리사무소팩스,
    세부.홈페이지주소,
    세부.단지분류,
    세부.일반관리방식,
    세부.일반관리인원,
    세부.일반관리계약업체,
    세부.경비관리방식,
    세부.경비관리인원,
    세부.경비관리계약업체,
    세부.청소관리방식,
    세부.청소관리인원,
    세부.음식물처리방법,
    세부.소독관리방식,
    세부.연간소독횟수,
    세부.소독방법,
    세부.건물구조,
    세부.세대전기계약방식,
    세부.화재수신반방식,
    세부.승강기관리형태,
    세부.지상주차장대수,
    세부.지하주차장대수,
    세부.cctv대수,
    세부.부대복리시설,
    세부.수전용량,
    세부.전기안전관리자법정선임여부,
    세부.급수방식,
    세부.승객용승강기,
    세부.화물용승강기,
    세부.비상승강기,
    세부.주차관제홈네트워크,
    세부.버스정류장,
    세부.편의시설,
    세부.교육시설,
    세부.복도유형,
    세부.사용승인일,
    세부.주거전용면적,
    세부.인근지하철역,
    세부.인근지하철호선,
    세부.지하철까지시간,
)
```

In [14]:

func writeAptInfo(열이름 []string, aptDetailInfo []AptDetailResponse) {
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("아파트정보", 열이름...))
if err != nil {
	fmt.Println(err)
}

for _, apt := range aptDetailInfo {
    
    매치 := apt.매치정보
    세부 := apt.세부정보
    
    // 세대정보와 주소가 null 로 응답되는 경우 처리
    if len(apt.세대정보) ==4 &&  len(apt.주소정보) ==2 {
     
    세대1 := apt.세대정보[0]
    세대2 := apt.세대정보[1]
    세대3 := apt.세대정보[2]
    세대4 := apt.세대정보[3]
    주소1 := apt.주소정보[0]
    주소2 := apt.주소정보[1]

    
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
    _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        주소1.법정동주소,
        주소2.법정동주소,
        세대1.세대수+세대2.세대수+세대3.세대수+세대4.세대수,
        세대1.세대수,
        세대2.세대수,
        세대3.세대수,
        세대4.세대수,
        세대1.아파트코드,
        세대2.아파트코드,
        세대3.아파트코드,
        세대4.아파트코드,
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
    }else if(len(apt.세대정보) ==4 &&  len(apt.주소정보) ==1){
        
        세대1 := apt.세대정보[0]
    세대2 := apt.세대정보[1]
    세대3 := apt.세대정보[2]
    세대4 := apt.세대정보[3]
    주소1 := apt.주소정보[0]

    
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
    _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        주소1.법정동주소,
        "",
        세대1.세대수+세대2.세대수+세대3.세대수+세대4.세대수,
        세대1.세대수,
        세대2.세대수,
        세대3.세대수,
        세대4.세대수,
        세대1.아파트코드,
        세대2.아파트코드,
        세대3.아파트코드,
        세대4.아파트코드,
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
}else {
            _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        "",
        "",
        0,
        0,
        0,
        0,
        0,
        "",
        "",
        "",
        "",
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
    }
        
	if err != nil {
		log.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}
    
}



# 최종 결과물 얻기

In [ ]:

createAptTable(createAptTableScript)

nameList := []string{"부천시", "시흥시", "광명시","구로구","양천구","부평구"}

for _, name := range nameList{
    aptDetailInfo := getAptBaseInfo(name)
    writeAptInfo(열이름, aptDetailInfo)
}

pq: relation "아파트정보" already exists법정동 목록 시작 -----------
[[41190101 경기도 부천시 원미동] [41190102 경기도 부천시 심곡동] [41190103 경기도 부천시 춘의동] [41190104 경기도 부천시 도당동] [41190105 경기도 부천시 약대동] [41190106 경기도 부천시 소사동] [41190107 경기도 부천시 역곡동] [41190108 경기도 부천시 중동] [41190109 경기도 부천시 상동] [41190110 경기도 부천시 소사본동] [41190111 경기도 부천시 심곡본동] [41190112 경기도 부천시 범박동] [41190113 경기도 부천시 괴안동] [41190114 경기도 부천시 송내동] [41190115 경기도 부천시 옥길동] [41190116 경기도 부천시 계수동] [41190117 경기도 부천시 오정동] [41190118 경기도 부천시 여월동] [41190119 경기도 부천시 작동] [41190120 경기도 부천시 원종동] [41190121 경기도 부천시 고강동] [41190122 경기도 부천시 대장동] [41190123 경기도 부천시 삼정동] [41190124 경기도 부천시 내동]]
법정동 목록 완료 -----------
41190101 아파트 목록 조회 시작 -----------
[{199906 4119010100 원미두산  A42083802 0 181713.98 443698.4 0 경기도 부천시 원미동} {199904 4119010100 원미풍림  A42083901 0 181488.05 443280.03 0 경기도 부천시 원미동}]
{"resultList":[{"KAPT_USEDATE":"199906","BJD_CODE":"4119010100","KAPT_NAME":"원미두산","OPEN_TERM":"","KAPT_CODE":"A42083802","OCCU_FIRST_DATE":0,"X":181713.98,"Y":443698.4,"ENERGY_B_COUNT":0,"B

2018/03/26 10:09:34 example.com 200 OK
2018/03/26 10:09:34 example.com 200 OK


{"resultMap_match":{"KAPT_CODE":"A42083802","TOWN_CODE":6904},"resultMap_kapt":{"CODE_HEAT":"개별난방","SUBWAY_STATION":"-","SUBWAY_LINE":"1호선, 7호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"도포식,분무식,기구설치식,독이식","KAPT_PE5AREA":0,"KAPTD_ECNTB":20,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"6","KAPTD_ECNTE":0,"CODE_ECON":"종합계약","KAPT_PPAREA":0,"KAPTD_CCCNT":"174","KAPTD_ECNTM":0,"KAPT_TEL":"032-653-4021","WELFARE_FACILITY":"관리사무소, 노인정, 보육시설, 주민공동시설, 어린이놀이터","KAPTD_ECNTP":0,"KAPT_FAX":"032-653-4022","KAPT_ACC_COMPANY":"피닉스","KAPT_MGR_CNT":"7","KAPT_BCOMPANY":"두산건설","CONVENIENT_FACILITY":"관공서(원미구청, 원미1동사무소) 병원(부천성가병원) 백화점(롯데백화점) 대형상가(원미상가) 공원(원미산공원, 부천종합운동장) 기타(원미시장)","CODE_DISINF":"위탁관리(총액관리제)","CODE_EMGR":"상주선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"719","CODE_EMAINT":"위탁관리","KAPT_MCAREA":0,"CODE_ELEV":"위탁관리(총액관리제)","KAPT_MEAREA":0,"CODE_NET":"무","KAPT_MHAREA":0,"KAPT_USEDATE":"1999-06-24","KAPTD_WTIMEBUS":"5분이내","KAPT_COD

2018/03/26 10:09:34 example.com 200 OK


[{201502 4119010200 가온펠리스  A10027593 0 180525.33 442720.16 0 경기도 부천시 심곡동} {201406 4119010200 부천역푸르지오시티  A10027856 0 180964.06 442582.94 0 경기도 부천시 심곡동}]
{"resultList":[{"KAPT_USEDATE":"201502","BJD_CODE":"4119010200","KAPT_NAME":"가온펠리스","OPEN_TERM":"","KAPT_CODE":"A10027593","OCCU_FIRST_DATE":0,"X":180525.33,"Y":442720.16,"ENERGY_B_COUNT":0,"BJD_NAME":"경기도 부천시 심곡동"},{"KAPT_USEDATE":"201406","BJD_CODE":"4119010200","KAPT_NAME":"부천역푸르지오시티","OPEN_TERM":"","KAPT_CODE":"A10027856","OCCU_FIRST_DATE":0,"X":180964.06,"Y":442582.94,"ENERGY_B_COUNT":0,"BJD_NAME":"경기도 부천시 심곡동"}]}
아파트 목록 조회 완료 -----------
아파트 목록 마샬링 시작 -----------
아파트 목록 마샬링 완료 -----------
아파트 정보 조회 시작 -----------


...

2018/03/26 10:13:48 example.com 200 OK


{"resultMap_match":{"KAPT_CODE":"A15805105","TOWN_CODE":3189},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"오목교, 신목동","SUBWAY_LINE":"5호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"-","KAPT_PE5AREA":0,"KAPTD_ECNTB":0,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"2","KAPTD_ECNTE":0,"CODE_ECON":"기타","KAPT_PPAREA":0,"KAPTD_CCCNT":"32","KAPTD_ECNTM":0,"KAPT_TEL":"02-2644-4736","WELFARE_FACILITY":"관리사무소, 노인정, 어린이놀이터","KAPTD_ECNTP":7,"KAPT_FAX":"02-6737-4736","KAPT_ACC_COMPANY":"-","KAPT_MGR_CNT":"4","KAPT_BCOMPANY":"성원건설","CONVENIENT_FACILITY":"관공서(목1동 주민센터) 병원(홍익병원) 백화점(현대백화점, 이마트, 홈플러스)","CODE_DISINF":"위탁관리","CODE_EMGR":"위탁선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"160","CODE_EMAINT":"위탁관리","KAPT_MCAREA":0,"CODE_ELEV":"위탁관리","KAPT_MEAREA":0,"CODE_NET":"무","KAPT_MHAREA":0,"KAPT_USEDATE":"1997-01-25","KAPTD_WTIMEBUS":"5분이내","KAPT_CODE":"A15805105","CODE_APT":"아파트","CODE_WSUPPLY":"고가수조식","KAPT_MPAREA":0,"KAPT_MAREA"

2018/03/26 10:13:49 example.com 200 OK


{"resultMap_match":{"KAPT_CODE":"A10027849","TOWN_CODE":20322330},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"-","SUBWAY_LINE":"5호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"도포식","KAPT_PE5AREA":0,"KAPTD_ECNTB":0,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"4","KAPTD_ECNTE":0,"CODE_ECON":"종합계약","KAPT_PPAREA":0,"KAPTD_CCCNT":"67","KAPTD_ECNTM":0,"KAPT_TEL":"02-2061-4087","WELFARE_FACILITY":"어린이놀이터, 휴게시설, 커뮤니티공간","KAPTD_ECNTP":6,"KAPT_FAX":"02-20614086","KAPT_ACC_COMPANY":"-","KAPT_MGR_CNT":"5","KAPT_BCOMPANY":"(주)대우건설","CONVENIENT_FACILITY":"-","CODE_DISINF":"위탁관리","CODE_EMGR":"상주선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"120","CODE_EMAINT":"자치관리","KAPT_MCAREA":0,"CODE_ELEV":"위탁관리","KAPT_MEAREA":0,"CODE_NET":"유","KAPT_MHAREA":0,"KAPT_USEDATE":"2015-06-30","KAPTD_WTIMEBUS":"5분이내","KAPT_CODE":"A10027849","CODE_APT":"주상복합","CODE_WSUPPLY":"부스타방식","KAPT_MPAREA":0,"KAPT_MAREA":22859.9006,"KAPT_PAREA":0,"KAPT_TARE

2018/03/26 10:13:49 example.com 200 OK


In [ ]:
type AptInfo struct {

    아파트코드 string `json:"아파트코드"`
    아파트이름 	string `json:"아파트이름"`
    마을코드 int `json:"마을코드"`
    법정동주소 string `json:"법정동주소"`
    도로명주소 string `json:"도로명주소"`
    단지분류	string `json:"단지분류"`
    
    동수 string `json:"동수"`
    세대수 int `json:"세대수"`
    
    전용면적60이하아파트코드 string  `json:"전용면적60이하아파트코드"`
    전용면적60이하세대수 string `json:"전용면적60이하세대수"`
    전용면적60이상85이하아파트코드 string `json:"전용면적60이상85이하아파트코드"`
    전용면적60이상85이하세대수 string `json:"전용면적60이상85이하세대수"`
    전용면적85이상135이하아파트코드 string `json:"전용면적85이상135이하아파트코드"`
    전용면적85이상135이하세대수 string `json:"전용면적85이상135이하세대수"`
    전용면적135초과아파트코드 string `json:"전용면적135초과아파트코드"`
    전용면적135초과세대수 string `json:"전용면적135초과세대수"`
    
    분양형태	string `json:"분양형태"`
    
    난방방식	string `json:"난방방식"`
    복도유형	string `json:"복도유형"`
    
    연면적	float64 `json:"연면적"`
    
    시공사 string `json:"시공사"`
    시행사 string `json:"시행사"`
    
    관리사무소연락처	string `json:"관리사무소연락처"`
    관리사무소팩스	string `json:"관리사무소팩스"`
    홈페이지주소	string `json:"홈페이지주소"`

    
    일반관리방식 string `json:"일반관리방식"`
    일반관리인원 	string `json:"일반관리인원"`
    일반관리계약업체  	string `json:"일반관리계약업체"` 
    
    경비관리방식	string `json:"경비관리방식"`
    경비관리인원	string `json:"경비관리인원"`
    경비관리계약업체 	string `json:"경비관리계약업체"`

    청소관리방식	string `json:"청소관리방식"`
    청소관리인원	string `json:"청소관리인원"`
    음식물처리방법 	string `json:"음식물처리방법"`
    
    소독관리방식	string `json:"소독관리방식"`
    연간소독횟수 	string `json:"연간소독횟수"`
    소독방법	string `json:"소독방법"`
    
    승강기관리형태  string `json:"승강기관리형태"`
    승객용승강기  string `json:"승객용승강기"`
    화물용승강기  string `json:"화물용승강기"`
    비상승강기 string `json:"비상승강기"`

        주차관제홈네트워크  	string `json:"주차관제홈네트워크"`
    
            건물구조  string `json:"건물구조"`
    세대전기계약방식 string `json:"세대전기계약방식"`
    화재수신반방식  string `json:"화재수신반방식"`
    
    지상주차장대수 string `json:"지상주차장대수"`
    지하주차장대수 string `json:"지하주차장대수"`
    
    cctv대수  string `json:"cctv대수"`
    부대복리시설  string `json:"부대복리시설"`
    수전용량  string `json:"수전용량"`
    전기안전관리자법정선임여부  	string `json전기안전관리자법정선임여부"`
    급수방식  string `json:"급수방식"`
    
    버스정류장 	string `json:"버스정류장"`
    편의시설  	string `json:"편의시설"`
    교육시설  	string `json:"교육시설"`
    
    사용승인일	string `json:"사용승인일"`
    주거전용면적 float64 `json:"주거전용면적"`
    
    인근지하철역	string `json:"인근지하철역"`
    인근지하철호선	string `json:"인근지하철호선"`
    지하철까지시간 	string `json:"지하철까지시간"`
 }


// 배열이나 json 으로 쉽게 덤프하고 싶은데 이상하게 비어서 나오네ㅡㅡ;;.

https://stackoverflow.com/questions/43692136/how-do-i-get-a-slice-from-a-postgres-array-in-golang

https://stackoverflow.com/questions/46675984/golang-postgres-how-to-dump-tables-into-json

http://sqlfiddle.com/#!15/5860d/4445

In [ ]:
&aptInfo.아파트코드,
&aptInfo.마을코드,
&aptInfo.법정동주소,
&aptInfo.도로명주소,
&aptInfo.세대수,
&aptInfo.전용면적60이하세대수,
&aptInfo.전용면적60이상85이하세대수,
&aptInfo.전용면적85이상135이하세대수,
&aptInfo.전용면적135초과세대수,
&aptInfo.전용면적60이하아파트코드,
&aptInfo.전용면적60이상85이하아파트코드,
&aptInfo.전용면적85이상135이하아파트코드,
&aptInfo.전용면적135초과아파트코드,
&aptInfo.아파트이름,
&aptInfo.분양형태,
&aptInfo.난방방식,
&aptInfo.연면적,
&aptInfo.동수,
&aptInfo.시공사,
&aptInfo.시행사,
&aptInfo.관리사무소연락처,
&aptInfo.관리사무소팩스,
&aptInfo.홈페이지주소,
&aptInfo.단지분류,
&aptInfo.일반관리방식,
&aptInfo.일반관리인원,
&aptInfo.일반관리계약업체,
&aptInfo.경비관리방식,
&aptInfo.경비관리인원,
&aptInfo.경비관리계약업체,
&aptInfo.청소관리방식,
&aptInfo.청소관리인원,
&aptInfo.음식물처리방법,
&aptInfo.소독관리방식,
&aptInfo.연간소독횟수,
&aptInfo.소독방법,
&aptInfo.건물구조,
&aptInfo.세대전기계약방식,
&aptInfo.화재수신반방식,
&aptInfo.승강기관리형태,
&aptInfo.지상주차장대수,
&aptInfo.지하주차장대수,
&aptInfo.cctv대수,
&aptInfo.부대복리시설,
&aptInfo.수전용량,
&aptInfo.전기안전관리자법정선임여부,
&aptInfo.급수방식,
&aptInfo.승객용승강기,
&aptInfo.화물용승강기,
&aptInfo.비상승강기,
&aptInfo.주차관제홈네트워크,
&aptInfo.버스정류장,
&aptInfo.편의시설,
&aptInfo.교육시설,
&aptInfo.복도유형,
&aptInfo.사용승인일,
&aptInfo.주거전용면적,
&aptInfo.인근지하철역,
&aptInfo.인근지하철호선,
&aptInfo.지하철까지시간,

In [31]:

import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
    "strings"
    

    // json encoding 을 위한 go 표준 라이브러리
    "encoding/json"

    "io/ioutil"

    "net/http"

)

func getAptInfoToJson(keyword string)  []AptInfo {
    
    _ = pg.Efatal
    
    const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
    )
 
    dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
        DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)
    
    // db 접속 포인터를 얻고
    db, err := sql.Open("postgres", dbinfo)

    if err != nil {
        log.Println(err)
    }
    defer db.Close()

    fmt.Println("포인터 얻기")

    
    // Query the database.
//     rows, err := db.Query(`SELECT * FROM 아파트정보 WHERE 법정동주소 LIKE $1`,
//                           "%"+ keyword+"%")

    // rows, err := db.Query(`SELECT json_agg(아파트정보) FROM 아파트정보`)
    rows, err := db.Query(`SELECT to_json(아파트정보) FROM 아파트정보`)
   
    if err != nil {
    log.Println(err)
    }
    defer rows.Close()

    aptInfos := []AptInfo{}
     aptInfo := AptInfo{}
    for rows.Next() {
        var jsonStr string
        rows.Scan(&jsonStr)
        if err := json.Unmarshal([]byte(jsonStr), &aptInfo); err != nil {
            log.Print(err)
        }
        fmt.Println(aptInfo)
        // fmt.Println(jsonStr)
        aptInfos = append(aptInfos,aptInfo)

    }
    
    return aptInfos
}

getAptInfoToJson("신길동")

포인터 얻기
{A42083802 원미두산 6904 경기도 부천시 원미동 151-1 두산아파트 경기도 부천시 원미로144번길 51 아파트 6 820         분양 개별난방 계단식 110940.465 두산건설 두산건설 032-653-4021                               0   }
{A42083901 원미풍림 6905 경기도 부천시 원미동 114-1 원미동풍림아파트 경기도 부천시 원미로 100 아파트 6 808         분양 개별난방 혼합식 1.03458922e+08 풍림건설(주) 풍림산업(주) 032-611-6276                               0   }
{A10027593 가온펠리스 20312892 경기도 부천시 심곡동 382-6 가온팰리스 경기도 부천시 부일로 453 도시형 생활주택 1 186         분양 개별난방 계단식 11957.76 OK건설 이제레저 032-216-0791                               0   }
{A10027856 부천역푸르지오시티 20294745 경기도 부천시 심곡동 145-6 부천역푸르지오시티 경기도 부천시 부일로 500 도시형 생활주택 1 162         분양 개별난방 복도식 5656.813 대우건설 주식회사 알이어드바이저스 주식회사 032-223-2202                               0   }
{A42085903 부천춘의 253317 경기도 부천시 춘의동 237 대한주택공사아파트 경기도 부천시 원미로 202 아파트 4 976         영구임대 중앙난방 복도식 43470.441 진덕산업 한국토지주택공사 0326112387                               0   }
{A42085801 춘의우남푸르미아 20047727 경기도 부천시 춘의동 141-2 우남 푸르미아 아파트 경기도 부천시 길주로377번길 13 아파트 5 268         분양 개별난방 계단식 34496.04 우남건설 우남건

{A42084903 중동중흥마을극동두산 7082 경기도 부천시 중동 1054 중흥마을 경기도 부천시 중동로280번길 63 아파트 10 516         분양 지역난방 계단식 90607 극동 두산건설 극동 두산 032-322-0865                               0   }
{A42084904 중동중흥마을신동아영남 7084 경기도 부천시 중동 1055 중흥마을 경기도 부천시 중동로280번길 27 아파트 11 640         분양 지역난방 계단식 115314.352 신동아영남 신동아영남 032-323-0537                               0   }
{A42084906 중동중흥마을주공 7086 경기도 부천시 중동 1052 중흥마을 경기도 부천시 계남로 196 아파트 7 863         분양 지역난방 복도식 57550.91 대한주택공사 대한주택공사 032-324-7401                               0   }
{A42002001 중동팰리스카운티 20108632   아파트 29 0         임대+분양 지역난방 계단식 429505.005 현대, 대우, 대림 중동 주공재건축조함 032-668-8400                               0   }
{A42085301 중동포도마을뉴서울 7087 경기도 부천시 중동 1171 중동신도시포도마을 경기도 부천시 소향로 124 아파트 6 310         분양 지역난방 계단식 48992.6 (주)뉴서울주택건설 (주)뉴-서울주택건설 032-322-1802                               0   }
{A42281104 부천한신 6848 경기도 부천시 소사본동 277-12 부천한신아파트 경기도 부천시 소사로170번길 81 아파트 9 916         분양 개별난방 혼합식 85735.364 한신공영 한신공영 032-346-2970                               0   }
{A420

{A42223003 부천소사역푸르지오 20165959 경기도 부천시 소사본동 133-12 부천소사역푸르지오 경기도 부천시 소삼로 47 아파트 9 797         임대+분양 개별난방 계단식 131821.0937 대우건설 골드c&d 032-346-1197                               0   }
{A42270610 소사SK-VIEW 253401 경기도 부천시 소사본동 135 소사에스케이뷰 경기도 부천시 소삼로 62 아파트 14 1172         분양 개별난방 계단식 162564.358 에스케이건설 조합 032-341-2734                               0   }
{A42281203 소사두산삼성 6842 경기도 부천시 소사본동 403 삼성.두산아파트 경기도 부천시 소사로78번길 81 아파트 10 524         분양 중앙난방 계단식 97542 두산 삼성 두산 삼성 032-348-7087                               0   }
{A42270606 소사양우 51097 경기도 부천시 소사본동 323 양우아파트 경기도 부천시 소사로126번길 18 아파트 3 244         분양 개별난방 복도식 25866.246 양우건설 양우지역조합 032-351-3500                               0   }
{A42281205 소사주공 6845 경기도 부천시 소사본동 400-8 주공아파트 경기도 부천시 소사로102번길 67 아파트 14 1210         분양 중앙난방 복도식 93175.71 삼익,화인 주택공사 032-214-8334                               0   }
{A42270609 소사주공2단지 20050439 경기도 부천시 소사본동 408-1 주공아파트 경기도 부천시 은성로 110-1 아파트 7 557         국민임대 개별난방 혼합식 45582.054 진흥기업 LH 0323515472                    

{A10026648 옥길호반베르디움아파트 20359154   아파트 16 0         분양 지역난방 계단식 227309.0137 호반건설 KB부동산신탁 0323460770                               0   }
{A10026650 제일풍경채 제이드카운티 2단지 20357903 경기도 부천시 옥길동 739-1 제이드카운티2단지 경기도 부천시 양지로 234-9 아파트 6 574         분양 지역난방 계단식 84512.76 제일건설 카우파트너스 0323433431                               0   }
{A42181402 오정세종2차 7016 경기도 부천시 오정동 179 세종아파트 경기도 부천시 오정로252번길 41 아파트 5 384         분양 개별난방 복도식 40525.55 세종건설(주) 세종건설(주) 032-676-6594                               0   }
{A42181401 오정창보 7020 경기도 부천시 오정동 373 창보아파트 경기도 부천시 오정로 245 아파트 5 290         분양 개별난방 복도식 34717 창보종합건설 창보종합건설 032-676-5364                               0   }
{A42117005 오정휴먼시아1단지 20059802 경기도 부천시 오정동 717 오정생활휴먼시아 경기도 부천시 오정로251번길 39 아파트 9 612         분양 지역난방 계단식 72670.632 LH 주공 0326760163                               0   }
{A42117003 오정휴먼시아2단지 20059803 경기도 부천시 오정동 726 오정생활휴먼시아 경기도 부천시 오정로 253 아파트 9 501         분양 지역난방 계단식 70391.742 이수건설 대한주택공사 032-671-0756                               0   }
{A42117004 오정휴먼시아

[{A42083802 원미두산 6904 경기도 부천시 원미동 151-1 두산아파트 경기도 부천시 원미로144번길 51 아파트 6 820         분양 개별난방 계단식 110940.465 두산건설 두산건설 032-653-4021                               0   } {A42083901 원미풍림 6905 경기도 부천시 원미동 114-1 원미동풍림아파트 경기도 부천시 원미로 100 아파트 6 808         분양 개별난방 혼합식 1.03458922e+08 풍림건설(주) 풍림산업(주) 032-611-6276                               0   } {A10027593 가온펠리스 20312892 경기도 부천시 심곡동 382-6 가온팰리스 경기도 부천시 부일로 453 도시형 생활주택 1 186         분양 개별난방 계단식 11957.76 OK건설 이제레저 032-216-0791                               0   } {A10027856 부천역푸르지오시티 20294745 경기도 부천시 심곡동 145-6 부천역푸르지오시티 경기도 부천시 부일로 500 도시형 생활주택 1 162         분양 개별난방 복도식 5656.813 대우건설 주식회사 알이어드바이저스 주식회사 032-223-2202                               0   } {A42085903 부천춘의 253317 경기도 부천시 춘의동 237 대한주택공사아파트 경기도 부천시 원미로 202 아파트 4 976         영구임대 중앙난방 복도식 43470.441 진덕산업 한국토지주택공사 0326112387                               0   } {A42085801 춘의우남푸르미아 20047727 경기도 부천시 춘의동 141-2 우남 푸르미아 아파트 경기도 부천시 길주로377번길 13 아파트 5 268         분양 개별난방 계단식 34496.04 우남건설 우남건설 032-

사 032-674-2386                               0   } {A42182803 여월휴먼시아5단지 20130297 경기도 부천시 여월동 316 여월휴먼시아 경기도 부천시 여월로 27 아파트 11 334         분양 지역난방 계단식 72016.33 요진건설외 다수 LH(토지주택공사) 032-678-3278                               0   } {A42122001 여월휴먼시아입주자대표 20088052 경기도 부천시 여월동 340 여월휴먼시아3단지 경기도 부천시 길주로573번길 37 아파트 13 899         분양 지역난방 계단식 122728.253 진흥, 풍림 LH공사 032-684-3138                               0   } {A42120101 세창짜임 253542 경기도 부천시 원종동 468 세창짜임아파트 경기도 부천시 소사로 855 아파트 3 270         분양 개별난방 계단식 35266.64 (주)세창 세창 032-682-6464                               0   } {A42120001 원종e-편한세상 20009576 경기도 부천시 원종동 469 원종e편한세상 경기도 부천시 삼작로368번길 15-12 아파트 2 158         분양 개별난방 계단식 20173.35 주)삼호 주)대림 032-671-1364                               0   } {A42120003 원종금호어울림아파트 20135499 경기도 부천시 원종동 470 원종동금호어울림 경기도 부천시 소사로819번길 33 아파트 9 404         임대+분양 개별난방 계단식 68136.79 금호건설 삼경아파트주택재건축정비사업조합 032-679-6262                               0   } {A42182701 원종주공 6955 경기도 부천시 원종동 129-19 원종2동 129-19 경기도 부천시 소사로807번길

https://stackoverflow.com/questions/46675984/golang-postgres-how-to-dump-tables-into-json

In [51]:

import (
	"database/sql"
	"log"
	"os"
    "fmt"
    "reflect"
    pg "github.com/lib/pq"
    "strings"
    

    // json encoding 을 위한 go 표준 라이브러리
    "encoding/csv"

    "io/ioutil"
"bufio"
    "net/http"

)

func getAptInfoToCsv(keyword string) {
    
    _ = pg.Efatal
    
    const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
    )
 
    dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
        DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)
    
    // db 접속 포인터를 얻고
    db, err := sql.Open("postgres", dbinfo)

    if err != nil {
        log.Println(err)
    }
    defer db.Close()

    fmt.Println("포인터 얻기")
    
    
    //open output file
    out, err := os.Create("./aptInfos.json")
    if err != nil {
        log.Println(err)
    }
    defer out.Close()

   
    rows, err := db.Query(`COPY (SELECT * FROM 아파트정보) TO STDOUT WITH CSV`)

    
    if err != nil {
        log.Println(err)
    }
    
//     writer := csv.NewWriter(bufio.NewWriter(out))
    writer := csv.NewWriter(out)
    defer writer.Flush()
    
    var csvStr string
    
    for rows.Next() {
        
        rows.Scan(&csvStr)
           
   
//          for _, value := range csvStr {
//              err := writer.Write(value)
            
//         }
    
        
        }

//     fmt.Println(csvStr)
}

getAptInfoToCsv("신길동")

포인터 얻기


2018/03/26 10:00:36 pq: unknown response for simple query: 'H'
panic: runtime error: invalid memory address or nil pointer dereference

goroutine 1055 [running]:
runtime/debug.Stack(0xc400000008, 0x7f2b94341490, 0xc420b43c70)
	/usr/local/go/src/runtime/debug/stack.go:24 +0xa9
github.com/yunabe/lgo/core.(*resultCounter).recordResult(0xc420b43c58, 0x7f2b942531a0, 0x7f2b94663460)
	/go/src/github.com/yunabe/lgo/core/core.go:91 +0xce
github.com/yunabe/lgo/core.(*resultCounter).recordResultInDefer(0xc420b43c58)
	/go/src/github.com/yunabe/lgo/core/core.go:96 +0x3b
panic(0x7f2b942531a0, 0x7f2b94663460)
	/usr/local/go/src/runtime/panic.go:491 +0x294
sync.(*RWMutex).RLocker(...)
	/usr/local/go/src/database/sql/sql.go:2447
database/sql.(*Rows).Next(0x0, 0xc4214d23b0)
	/usr/local/go/src/database/sql/sql.go:2447 +0x32
github.com/yunabe/lgo/sess7b2274696d65223a313532323035303830393336373539333539307d/exec51.LgoExport_getAptInfoToCsv(0x7f2b63b09299, 0x9)
	/go/src/github.com/yunabe/lgo/sess7b2274696d6